In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd/content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [4]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 20


In [5]:
# Load the labels from the Excel sheet
labels_df = pd.read_csv('./miml_dataset/miml_labels_2.csv')
labels_df.head()


,Filenames,labels
0,1.jpg,desert
1,2.jpg,desert
2,3.jpg,desert
3,4.jpg,"desert,mountains"
4,5.jpg,desert


In [6]:
print(labels_df)


     Filenames            labels
0        1.jpg            desert
1        2.jpg            desert
2        3.jpg            desert
3        4.jpg  desert,mountains
4        5.jpg            desert
...        ...               ...
1995  1996.jpg             trees
1996  1997.jpg             trees
1997  1998.jpg             trees
1998  1999.jpg             trees
1999  2000.jpg             trees

[2000 rows x 2 columns]


In [7]:
print(labels_df.columns)


Index(['Filenames', 'labels'], dtype='object')


In [8]:

# Check the column names in the DataFrame
print(labels_df.columns)

Index(['Filenames', 'labels'], dtype='object')


In [9]:
# Define the ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [10]:
num_classes = len(labels_df['labels'].unique())

In [11]:
train_set = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory='./miml_dataset/images',
    x_col='Filenames',
    y_col='labels',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42,
    validate_filenames=True,
    interpolation='bilinear',
    color_mode='rgb',
    save_format='jpg',
    num_classes=num_classes)

Found 1600 validated image filenames belonging to 20 classes.


In [12]:
# Load the validation set
val_set = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory='./miml_dataset/images',
    x_col='Filenames',
    y_col='labels',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

Found 400 validated image filenames belonging to 20 classes.


In [13]:
num_classes = len(labels_df['labels'].unique())


In [14]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten



In [15]:
# Define the CNN model architecture
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(len(labels_df['labels'].unique()), activation='sigmoid'))

In [16]:
# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [17]:
# Train the model
history = model.fit(
    train_set,
    steps_per_epoch=train_set.samples // BATCH_SIZE,
    validation_data=val_set,
    validation_steps=val_set.samples // BATCH_SIZE,
    epochs=30)

Epoch 1/30
50/50 [==============================] - 898s 18s/step - loss: 0.1576 - accuracy: 0.3187 - val_loss: 0.4012 - val_accuracy: 0.0000e+00
Epoch 2/30
50/50 [==============================] - 213s 4s/step - loss: 0.1103 - accuracy: 0.4825 - val_loss: 0.3884 - val_accuracy: 0.0000e+00
Epoch 3/30
50/50 [==============================] - 205s 4s/step - loss: 0.1007 - accuracy: 0.5562 - val_loss: 0.4637 - val_accuracy: 0.0000e+00
Epoch 4/30
50/50 [==============================] - 203s 4s/step - loss: 0.0947 - accuracy: 0.5825 - val_loss: 0.4538 - val_accuracy: 0.0000e+00
Epoch 5/30
50/50 [==============================] - 203s 4s/step - loss: 0.0930 - accuracy: 0.5850 - val_loss: 0.4381 - val_accuracy: 0.0000e+00
Epoch 6/30
50/50 [==============================] - 198s 4s/step - loss: 0.0896 - accuracy: 0.6012 - val_loss: 0.4589 - val_accuracy: 0.0000e+00
Epoch 7/30
50/50 [==============================] - 200s 4s/step - loss: 0.0816 - accuracy: 0.6506 - val_loss: 0.4796 - val_accur